# Exercise 1.2:  Queue Theory (Part 3)

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>
 
---

In [ ]:
#@title
# Import libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

We've got a flexible code that let's us test our different queueing systems, and can pull out useful fairly consistent metrics from our simulations, despite the fact that it is a highly variable stochastic system.

<br> 

But what if we want to get a bigger picture of how well the systems work in different situations (i.e. different arrival and departure rates).  Let's say you know the arrival time in the evenings is higher, and you want to know how many staff you'll need in the store to keep the average wait time below a certain level.  We'll want to sweep our parameters, right?   So in this last section, let's set up our code to do just that.

<br>

I've included code here that we developed earlier.  But if you'd like to use the code you developed in part 2 here, feel free to replace these.  

In [ ]:
# Define new run_simulation and mult_runs
def mult_runs(lam,mu,num_sims,num_steps,change_func):
    mean = pd.Series([],dtype = object)
    for i in range(num_sims):
        line = pd.Series(dict(cust=0,cust2=0),name='Customers in line')
        global_results = run_simulation(line, lam,mu,num_steps, change_func)
        metrics = compute_metrics(global_results,lam)
        mean[i] = metrics[0]
    return mean

def run_simulation(state,lam,mu,num_steps,change_func):
    """Simulate a queueing system.
    """

    results = pd.Series([],dtype=object)
    results[0] = state.sum()
    
    for t in range(num_steps):
        state = change_func(state,lam,mu)
        results[t+1] = state.sum()
        
    results.plot(label='customers in line',color='C3',
             title='Customers in line',xlabel='time',
             ylabel='Number of customers');
    return results

def compute_metrics(results, lam):
    L = results.mean()
    M = results.max()
    W = L / lam
    #print("The average number of customers in line is",f'{L:.3}', "customers.")
    #print("The maximum number of customers in line is", M, "customers.")
    #print("The average time spent by customers in line is", f'{W:.3}', "minutes.")

    return L, M, W

In [ ]:
# One queue and one server
def change_func(state, lam, mu):
    # Check to see if there are any customers in line
    if state.cust > 0:
        # If there are customers in line, check to see if one leaves
        if flip(mu):
            state.cust -= 1
            
    # The addition of new custormers happens second.
    # This assumes that a newly arrived customer can not be served in 
    # the minute that s/he arrives.
    if flip(lam):
        state.cust += 1
    return state
    
# One queue and two servers
def change_func_1q_2s(state,lam,mu):
    # Check to see if either checker finishes with a customer
    if state.cust > 0:
        if flip(mu):
            state.cust -= 1
    if state.cust > 0:
        if flip(mu):
            state.cust -= 1
    # The addition of new custormers happens second.
    # This assumes that a newly arrived customer can not be served in 
    # the minute that s/he arrives.
    if flip(lam):
        state.cust += 1
        
    return state
    
# Two queues and two servers
def change_func_2q_2s(state,lam,mu):
 
    if state.cust > 0:
        if flip(mu):
            state.cust -= 1
    if state.cust2 > 0:
        if flip(mu):
            state.cust2 -= 1
            
    # The following assumes that the arriving customer will 
    # join the shortest line
    if flip(lam):
        if state.cust > state.cust2:
            state.cust2 += 1
        else:
            state.cust += 1

    
    return state

In [ ]:
lam = 0.5
mu = 0.5
num_sims = 25
num_steps = 120
mean = mult_runs(lam, mu, num_sims, num_steps, change_func_1q_2s)
mean.mean()

### Part 8: Sweeping Parameters

We want to understand how the store responds to different arrival rates, so we will sweep through possible values for $\lambda$.  If we sweep through values for $\lambda$ (the arrival rate), we want to consider only values that are realistic. Consider, for example, that if customers arrive faster than the completion rate (that is, $\lambda > \mu$), the queue tends to grow without bound.  In that case the metrics `L` and `W` just depend on how long the store is open, and the result isn't very interesting.  Likewise, if $\lambda$ is much less than $\mu$, the store essentially stays empty: again, not very interesting.

<br>

So we want values that are a significant fraction of $\mu$.   To keep our model as general as we can (to allow for changes in $\mu$, let's sweep through a range of possibilities defined by $\mu$.  We'll set the simulation to sweep through $\lambda$ values that range from 20% to 90% of the departure rate, $\mu$. 

<br>

To do that, we'll need a range of values from `0.2*mu` to `0.9*mu`.  Fortunately, there's a NumPy function called `linspace` that can do that for us:

In [ ]:
from numpy import linspace

p1_array = linspace(0, 1, 5)
p1_array

The arguments indicate:
* where the sequence should start
* where the sequence should stop
* how many elements it should contain. 

In this example, the sequence contains
`5` equally-spaced numbers, starting at `0` and ending at `1`.  The sequence is held in a NumPy *array*, which is a container for a sequence of numbers.

<br>

This is useful for `For` loops.  Now, instead of writing `for i in range(num_steps)` to start a loop, we can write `for i in lam_array`.  

In [ ]:
# Create values for lam
mu = 0.5
lam_array = linspace(0.2*mu,0.9*mu,15)

for i in lam_array:
    print(i)


The loop will run 15 times, but `i`, instead of being `0,1,2,3...`, will be equal to `0.1, 0.125, 0.15...`.

<br>

Now let's use `linspace`.  Write a function `sweep_func` that takes `state`, an array of values for `lam`, a single value for `mu`, `num_sims`, `num_steps`, and a change function.  The function should:

* create an empty Series called `sweep`

* Loop through the values in `lam`: for each value of `lam` it should run `mult_runs` and store the average value of `L` in `sweep`.

* "Return" the series `sweep`.

In [ ]:
# Define sweeping function
def sweep_func(lam,mu,num_sims,num_steps,change_func):


        

Call your function to generate a sweep `Series` for the single queue/ single server model, and plot it.  Two notes:

* The `plot` line in run_simulation is commented out here to give us a clean plot.
* Depending on how many lambda values you are sweeping through, this might take a couple minutes.  You can tell that a calculation is in process by noting the "Executing...." messages at the bottom of the screen.

In [ ]:
def run_simulation(state,lam,mu,num_steps,change_func):
    """Simulate a queueing system.
    """

    results = pd.Series([],dtype=object)
    results[0] = state.sum()
    
    for t in range(num_steps):
        state = change_func(state,lam,mu)
        results[t+1] = state.sum()
        
    #results.plot(label='customers in line',color='C3',
    #         title='Customers in line',xlabel='time',
    #         ylabel='Number of customers');
    return results

# Generate solution and plot
mu = 0.5
lam_array = linspace(0.2*mu,0.9*mu,15)
num_sims = 25
num_steps = 120

## Call sweep_func and plot the results Series


If we imagine that this range of values represents arrival rates on at different times of the day. If we wanted to ensure that the average number of customers in line at a given time stayed below 1.5, at what arrival rate would we need to open a second check out counter?

### Part 9: Comparing two check-out systems

Let's look at the two more complex systems, and compare how they react to different parameters.  Since we have two checkout counters now, we can consider values for $\lambda$ that exceed $\mu$.  Create a new array of values for `lam` from 40% to 180% of `mu`.

In [ ]:
# Create new array
mu=0.5


Use `sweep_function` to sweep these values for both two-checkout scenarios, and plot the average values of `L` for both of them on the same graph.

In [ ]:
# Generate solution and plot
num_sims = 25
num_steps = 120



### Part 10: Analytical Solutions

The $\lambda - \mu $ model used here is a common model in queueing theory, in part because many of its properties can be derived analytically.  In particular, we can derive the average number of customers in the store as a function of $\mu$ and $\lambda$ for the single queue/ single checkout model:

$$L = \mu / (\mu - \lambda)$$

✅ Look at that equation.  What happens as $\mu$ gets much larger than $\lambda$?  What does this mean in terms of the physical system (i.e. customers in line)?  What happens as $\lambda$ approaches the value of $\mu$?  Write your answers in a textbook before you run the cells below.

The following function plots the theoretical value of $L$ as a function of $\lambda$.

In [ ]:
def plot_L(lam_array, mu):
    """Plot the theoretical mean wait time.
    
    lam_array: array of values for `lam`
    mu: probability of finishing a checkout
    """
    L_array = lam_array / (mu - lam_array)
    # Transform this array into a pd.Series
    L_series = pd.Series(data=L_array, index=lam_array)
    L_series.plot(style='-', label='analysis',legend=True)

Let's use this function to plot the theoretical results, then plot your simulation results again on the same graph.  How do they compare?

In [ ]:
# Compare simulation to theoretical results on a plot
mu = 0.5
lam_array = linspace(0.2*mu,0.9*mu,15)
plot_L(lam_array,mu)
sweep_results.plot(label="simulation", 
                   title='Comparison of theoretical and simulation results',
                   xlabel='Arrival rate alpha',ylabel='Average Number of Customers in Line',
                  legend=True);

✅ The two plots diverge as $\lambda$ approaches $\mu$.  Why do you think that might be?  If $\lambda$ is close to or larger than $\mu$, what would happen to the size of the line as the time got much larger than 120 steps/minutes?  Write your answer in a sentence or two below this.